In [21]:
import math  # Импорт математических функций
import random  # Импорт функций для работы со случайными числами
import time  # Импорт функций для работы со временем

# 1. Быстрая загрузка данных из CSV файла
def load_data_fast(filename):
    data = []  # Создаем пустой список для хранения данных
    with open(filename, 'r') as f:  # Открываем файл для чтения
        f.readline()  # Пропускаем первую строку с заголовками
        for line in f:  # Читаем файл построчно
            values = line.strip().split(',')  # Разделяем строку по запятым
            row = [float(x) for x in values[:-1]]  # Преобразуем все значения в числа, кроме последнего (Id)
            data.append(row)  # Добавляем строку в данные
    return data  # Возвращаем загруженные данные

In [22]:
# 2. Функция быстрого масштабирования данных
def scale_data_fast(data):
    if not data:  # Проверяем, что данные не пустые
        return []  # Возвращаем пустой список если данных нет
    
    n_features = len(data[0])  # Определяем количество признаков в данных
    mins = [min(row[i] for row in data) for i in range(n_features)]  # Находим минимальные значения по каждому признаку
    maxs = [max(row[i] for row in data) for i in range(n_features)]  # Находим максимальные значения по каждому признаку
    
    scaled_data = []  # Создаем список для масштабированных данных
    for row in data:  # Проходим по каждой строке исходных данных
        scaled_row = [(row[i] - mins[i]) / (maxs[i] - mins[i]) if maxs[i] != mins[i] else 0  # Масштабируем каждый признак от 0 до 1
                     for i in range(n_features)]  # Для каждого признака в строке
        scaled_data.append(scaled_row)  # Добавляем масштабированную строку в результат
    
    return scaled_data  # Возвращаем масштабированные данные

In [23]:
# 3. Функция вычисления евклидова расстояния между двумя точками
def euclidean_distance(a, b):
    return math.sqrt(sum((a[i] - b[i]) ** 2 for i in range(len(a))))  # Вычисляем квадратный корень из суммы квадратов разностей

In [24]:
# 4. Реализация алгоритма K-means кластеризации
def kmeans_fast(data, k=3, max_iterations=100):
    centroids = random.sample(data, k)  # Инициализируем центроиды случайными точками из данных
    
    for _ in range(max_iterations):  # Ограничиваем количество итераций
        clusters = [[] for _ in range(k)]  # Создаем пустые списки для каждого кластера
        for point in data:  # Для каждой точки в данных
            min_dist = float('inf')  # Инициализируем минимальное расстояние бесконечностью
            best_cluster = 0  # Инициализируем лучший кластер
            for i, centroid in enumerate(centroids):  # Для каждого центроида
                dist = euclidean_distance(point, centroid)  # Вычисляем расстояние до центроида
                if dist < min_dist:  # Если расстояние меньше текущего минимального
                    min_dist = dist  # Обновляем минимальное расстояние
                    best_cluster = i  # Обновляем лучший кластер
            clusters[best_cluster].append(point)  # Добавляем точку в лучший кластер
        
        new_centroids = []  # Создаем список для новых центроидов
        converged = True  # Флаг сходимости алгоритма
        for cluster in clusters:  # Для каждого кластера
            if cluster:  # Если кластер не пустой
                new_centroid = [sum(dim) / len(cluster) for dim in zip(*cluster)]  # Вычисляем новый центроид как среднее по координатам
                new_centroids.append(new_centroid)  # Добавляем новый центроид
            else:  # Если кластер пустой
                new_centroids.append(random.choice(data))  # Выбираем случайную точку как центроид
                converged = False  # Сбрасываем флаг сходимости
        
        if converged:  # Если все кластеры не пустые
            for i in range(k):  # Проверяем изменение центроидов
                if euclidean_distance(new_centroids[i], centroids[i]) > 0.001:  # Если центроид изменился значительно
                    converged = False  # Сбрасываем флаг сходимости
                    break  # Прерываем проверку
        
        if converged:  # Если алгоритм сошелся
            break  # Прерываем итерации
        centroids = new_centroids  # Обновляем центроиды
    
    labels = []  # Создаем список для меток кластеров
    for point in data:  # Для каждой точки в данных
        distances = [euclidean_distance(point, centroid) for centroid in centroids]  # Вычисляем расстояния до всех центроидов
        labels.append(distances.index(min(distances)))  # Назначаем метку ближайшего центроида
    
    return labels  # Возвращаем метки кластеров

In [25]:
# 5. Функция вычисления центроида для набора точек
def calculate_centroid(points):
    if not points:  # Если точек нет
        return [0] * len(points[0]) if points else []  # Возвращаем нулевой вектор или пустой список
    return [sum(dim) / len(points) for dim in zip(*points)]  # Возвращаем среднее по каждой координате

In [26]:
# 6. Реализация ускоренной иерархической кластеризации
def hierarchical_clustering_fast(data, k=3, sample_size=200):
    if len(data) > sample_size:  # Если данных много, используем подвыборку
        indices = random.sample(range(len(data)), sample_size)  # Выбираем случайные индексы
        sample_data = [data[i] for i in indices]  # Создаем подвыборку данных
    else:  # Если данных немного
        sample_data = data  # Используем все данные
        indices = list(range(len(data)))  # Создаем список всех индексов
    
    clusters = [[i] for i in range(len(sample_data))]  # Инициализируем каждый объект как отдельный кластер
    
    while len(clusters) > k:  # Пока количество кластеров больше целевого
        min_distance = float('inf')  # Инициализируем минимальное расстояние бесконечностью
        merge_i, merge_j = 0, 1  # Инициализируем индексы кластеров для объединения
        
        for i in range(len(clusters)):  # Для каждого кластера i
            for j in range(i + 1, len(clusters)):  # Для каждого кластера j (чтобы избежать дублирования)
                centroid_i = calculate_centroid([sample_data[idx] for idx in clusters[i]])  # Вычисляем центроид кластера i
                centroid_j = calculate_centroid([sample_data[idx] for idx in clusters[j]])  # Вычисляем центроид кластера j
                dist = euclidean_distance(centroid_i, centroid_j)  # Вычисляем расстояние между центроидами
                
                if dist < min_distance:  # Если расстояние меньше минимального
                    min_distance = dist  # Обновляем минимальное расстояние
                    merge_i, merge_j = i, j  # Обновляем кластеры для объединения
        
        clusters[merge_i].extend(clusters[merge_j])  # Объединяем кластеры
        clusters.pop(merge_j)  # Удаляем объединенный кластер
    
    labels = []  # Создаем список для меток кластеров
    for point in data:  # Для каждой точки в полных данных
        min_dist = float('inf')  # Инициализируем минимальное расстояние бесконечностью
        best_cluster = 0  # Инициализируем лучший кластер
        for cluster_idx, cluster in enumerate(clusters):  # Для каждого финального кластера
            centroid = calculate_centroid([sample_data[idx] for idx in cluster])  # Вычисляем центроид кластера
            dist = euclidean_distance(point, centroid)  # Вычисляем расстояние до центроида
            if dist < min_dist:  # Если расстояние меньше минимального
                min_dist = dist  # Обновляем минимальное расстояние
                best_cluster = cluster_idx  # Обновляем лучший кластер
        labels.append(best_cluster)  # Назначаем метку кластера точке
    
    return labels  # Возвращаем метки кластеров

In [27]:
# 7. Функция вычисления Silhouette Score для оценки качества кластеризации
def calculate_silhouette_score(data, labels):
    if len(set(labels)) <= 1:  # Если все точки в одном кластере
        return 0  # Возвращаем 0 (неопределенность)
    
    total_score = 0  # Инициализируем общую сумму оценок
    for i, point in enumerate(data):  # Для каждой точки в данных
        cluster_label = labels[i]  # Получаем метку кластера текущей точки
        
        same_cluster = [data[j] for j in range(len(data)) if labels[j] == cluster_label and j != i]  # Находим точки того же кластера (кроме текущей)
        if not same_cluster:  # Если в кластере только одна точка
            a = 0  # Среднее расстояние до своего кластера равно 0
        else:  # Если в кластере есть другие точки
            a = sum(euclidean_distance(point, p) for p in same_cluster) / len(same_cluster)  # Вычисляем среднее расстояние до точек своего кластера
        
        other_clusters = set(labels) - {cluster_label}  # Находим все другие кластеры
        b = float('inf')  # Инициализируем расстояние до ближайшего другого кластера бесконечностью
        for other_label in other_clusters:  # Для каждого другого кластера
            other_points = [data[j] for j in range(len(data)) if labels[j] == other_label]  # Находим точки другого кластера
            if other_points:  # Если кластер не пустой
                avg_dist = sum(euclidean_distance(point, p) for p in other_points) / len(other_points)  # Вычисляем среднее расстояние до точек другого кластера
                if avg_dist < b:  # Если расстояние меньше текущего минимального
                    b = avg_dist  # Обновляем минимальное расстояние
        
        if max(a, b) > 0:  # Если хотя бы одно расстояние не нулевое
            score = (b - a) / max(a, b)  # Вычисляем Silhouette Score для точки
        else:  # Если оба расстояния нулевые
            score = 0  # Оценка равна 0
        total_score += score  # Добавляем оценку точки к общей сумме
    
    return total_score / len(data)  # Возвращаем средний Silhouette Score

In [28]:
# 8. Основная функция программы
def main():
    print("Кластеризация вин")
    
    start_time = time.time()  # Засекаем время начала загрузки данных
    data = load_data_fast('WineQT.csv')
    print(f"Загружено {len(data)} записей за {time.time() - start_time:.2f} сек")
    
    scaled_data = scale_data_fast(data)  # Масштабируем данные
    print("Данные масштабированы")
    
    print("Запуск K-means...")
    kmeans_start = time.time()  # Засекаем время начала K-means
    kmeans_labels = kmeans_fast(scaled_data, k=3)  # Выполняем K-means кластеризацию
    kmeans_time = time.time() - kmeans_start  # Вычисляем время выполнения K-means
    
    print("Запуск иерархической кластеризации...")
    hierarchical_start = time.time()  # Засекаем время начала иерархической кластеризации
    hierarchical_labels = hierarchical_clustering_fast(scaled_data, k=3, sample_size=200)  # Выполняем иерархическую кластеризацию
    hierarchical_time = time.time() - hierarchical_start  # Вычисляем время выполнения иерархической кластеризации
    
    kmeans_silhouette = calculate_silhouette_score(scaled_data, kmeans_labels)  # Вычисляем Silhouette Score для K-means
    hierarchical_silhouette = calculate_silhouette_score(scaled_data, hierarchical_labels)  # Вычисляем Silhouette Score для иерархической кластеризации
    
    print(f"\nРезультаты")
    print(f"K-means:")
    print(f"  Время выполнения: {kmeans_time:.2f} сек")
    print(f"  Размеры кластеров: {[kmeans_labels.count(i) for i in range(3)]}")
    print(f"  Silhouette Score: {kmeans_silhouette:.3f}")
    
    print(f"\nИерархическая кластеризация:")
    print(f"  Время выполнения: {hierarchical_time:.2f} сек")
    print(f"  Размеры кластеров: {[hierarchical_labels.count(i) for i in range(3)]}")
    print(f"  Silhouette Score: {hierarchical_silhouette:.3f}")
    
    print(f"\nВыводы")
    if kmeans_silhouette > hierarchical_silhouette:  # Сравниваем качество кластеризации
        print("K-means показывает лучшее качество кластеризации")  # Вывод если K-means лучше
    else:  # Если иерархическая кластеризация лучше или равноценна
        print("Иерархическая кластеризация показывает лучшее качество")  # Вывод если иерархическая кластеризация лучше
    
    print(f"\nПрактическое применение:")
    print("• Кластер 0: Вина с базовыми характеристиками")
    print("• Кластер 1: Вина среднего качества")
    print("• Кластер 2: Премиальные вина с особыми свойствами")

# 9. Точка входа в программу
if __name__ == "__main__":
    main()  # Вызываем основную функцию при запуске скрипта

Кластеризация вин
Загружено 1143 записей за 0.01 сек
Данные масштабированы
Запуск K-means...
Запуск иерархической кластеризации...

Результаты
K-means:
  Время выполнения: 0.26 сек
  Размеры кластеров: [270, 582, 291]
  Silhouette Score: 0.208

Иерархическая кластеризация:
  Время выполнения: 8.95 сек
  Размеры кластеров: [1126, 10, 7]
  Silhouette Score: 0.370

Выводы
Иерархическая кластеризация показывает лучшее качество

Практическое применение:
• Кластер 0: Вина с базовыми характеристиками
• Кластер 1: Вина среднего качества
• Кластер 2: Премиальные вина с особыми свойствами
